<a href="https://colab.research.google.com/github/hyogyeong/ESAA/blob/main/1004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install surprise

     |████████████████████████████████| 11.8 MB 47 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619398 sha256=432b56fef533cf7b12069a2114caec599f2a52d749dd772e2975fa44bad7fb9b
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving ratings_Electronics (1).csv to ratings_Electronics (1).csv


In [11]:
import pandas as pd
import numpy as np

In [12]:
import torch
from fastai import * 

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

In [13]:
electronics_data = pd.read_csv("ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])

NameError: ignored

In [ ]:
electronics_data.head()

In [ ]:
electronics_data.shape

In [ ]:
eletronics_data = electronics_data.iloc[:1048576,0:]

In [ ]:
eletronics_data.dtypes

In [ ]:
eletronics_data.describe()['Rating'].T

In [ ]:
print("max of rating : %d" %(eletronics_data.Rating.max()))
print("min of rating : %d" %(eletronics_data.Rating.min()))

Missig Values 수

In [ ]:
print('Number of missing values across columns: \n',electronics_data.isnull().sum())

Ratings

In [ ]:
with sns.axes_style('white'):
    g = sns.factorplot("Rating", data  = eletronics_data, aspect = 2.0, kind = 'count')
    g.set_ylabels("Total number of ratings")

In [ ]:
print("Total data")
print("-"*50)
print("\nTotal num of ratings :", eletronics_data.shape[0])
print("Total num of Users :", len(np.unique(eletronics_data.userId)))
print("Total num of products: ", len(np.unique(eletronics_data.productId)))

In [ ]:
eletronics_data.drop(['timestamp'],axis = 1, inplace = True)

Ratings 분석

In [ ]:
num_of_rated_products_per_user = eletronics_data.groupby(by = 'userId')['Rating'].count().sort_values(ascending=False)
num_of_rated_products_per_user.head()

In [ ]:
num_of_rated_products_per_user.describe()

거의 모든 유저들이 하나의 상품에만 평가한 것을 알 수 있다.

In [ ]:
quantiles = num_of_rated_products_per_user.quantile(np.arange(0,1.01,0.01), interpolation='higher')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Quantiles and their Values")
quantiles.plot()
# quantiles with 0.05 difference
plt.scatter(x=quantiles.index[::5], y=quantiles.values[::5], c='orange', label="quantiles with 0.05 intervals")
# quantiles with 0.25 difference
plt.scatter(x=quantiles.index[::25], y=quantiles.values[::25], c='m', label = "quantiles with 0.25 intervals")
plt.ylabel('No of ratings by user')
plt.xlabel('Value at the quantile')
plt.legend(loc='best')
plt.show()

In [ ]:
print('\n No of rated product more than 50 per user : {}\n'.format(sum(num_of_rated_products_per_user >= 50)) )

50개 이상의 상품을 평가한 유저 : 38명

Popularity Based Recommendation

In [ ]:
new_df=electronics_data.groupby("productId").filter(lambda x:x['Rating'].count() >=50)

평가가 50번 이상 된 상품들을 가져옴

In [ ]:
new_df.head()

In [ ]:
new_df.shape

In [ ]:
num_of_ratings_per_product = new_df.groupby(by='productId')['Rating'].count().sort_values(ascending=False)

fig = plt.figure(figsize=plt.figaspect(.5))
ax = plt.gca()
plt.plot(num_of_ratings_per_product.values)
plt.title('# RATINGS per Product')
plt.xlabel('Product')
plt.ylabel('No of ratings per product')
ax.set_xticklabels([])

plt.show()

각 상품 아이디별로 평가를 평균 내기

In [ ]:
new_df.groupby('productId')['Rating'].mean().head()

sorting

In [ ]:
new_df.groupby('productId')['Rating'].mean().sort_values(ascending=False).head()

각 상품별 평가 받은 수 기준으로 내림차순 정렬

In [ ]:
new_df.groupby('productId')['Rating'].count().sort_values(ascending=False).head()

In [ ]:
ratings_mean_count = pd.DataFrame(new_df.groupby('productId')['Rating'].mean())

In [ ]:
ratings_mean_count.head()

In [ ]:
ratings_mean_count['rating_counts'] = pd.DataFrame(new_df.groupby('productId')['Rating'].count())

In [ ]:
ratings_mean_count['rating_counts'].max()

In [ ]:
plt.figure(figsize=(6,6))
plt.rcParams['patch.force_edgecolor']=True
ratings_mean_count['rating_counts'].hist(bins=50)

In [ ]:
plt.figure(figsize=(8,6))
plt.rcParams['patch.force_edgecolor']=True
ratings_mean_count['Rating'].hist(bins=50)

In [ ]:
plt.figure(figsize=(8,6))
plt.rcParams['patch.force_edgecolor'] = True
sns.jointplot(x='Rating', y='rating_counts', data=ratings_mean_count, alpha=0.4)

In [ ]:
popular_products = pd.DataFrame(new_df.groupby('productId')['Rating'].count())
most_popular = popular_products.sort_values("Rating", ascending=False)
most_popular.head(30).plot(kind = "bar")

Collaborative Filtering 이용

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

1) Data 가져오기

In [ ]:
new_df.drop(['timestamp'],axis = 1, inplace = True)

In [ ]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(new_df,reader)

2) Trainset, Testset으로 나누기

In [ ]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state=10)

3) KNN을 이용 trainset에 파라미터 맞추기

In [ ]:
algo = KNNWithMeans(k=5, sim_options = {'name': 'pearson_baseline', 'user_based':False})
algo.fit(trainset)

4) Test accuracy 측정

In [ ]:
test_pred = algo.test(testset)

In [ ]:
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Model-based collaborative filtering syste

대량의 상품, 많은 사용자들에게 추천 가능, sparse한 matrices에도 사용가능

In [ ]:
new_df1 = new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values ='Rating', index ='userId', columns ='productId',fill_value =0)
ratings_matrix.head()

In [ ]:
ratings_matrix.shape

In [ ]:
X = ratings_matrix.T
X.head()

In [ ]:
X.shape

In [ ]:
X.index

In [ ]:
X1 = X

In [ ]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

In [ ]:
decomposed_matrix

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

In [ ]:
X.index[75]

In [ ]:
i = X.index[75]

product_names = list(X.index)
product_ID = product_names.index(i) #index구하기
product_ID

In [ ]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

In [ ]:
correlation_product_ID

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.65]) #피어슨 상관계수가 0.65보다 큰 상품들의 index의 list return

Recommend.remove(i) # 이미 구입한 상품은 뺀다

Recommend[0:24] #상위 24개 추천

In [ ]:
Recommend